In [1]:
# use 100% human data as the benchmark for all these studies!
try:
    import rbo
except:
    ! pip install rbo
    import rbo

In [2]:
import os
import json
import pandas as pd
import numpy as np
if os.getcwd() != '/home/jupyter/crisp':
    os.chdir("..")
from utils.gcp_helpers import get_dataframe_from_bucket
%pwd

'/home/jupyter/crisp'

In [3]:
import json

In [46]:
with open('results/GSE1_only_file/results_for_bucket.json') as json_file:
    base = json.load(json_file)
with open('results/one_norm_file/results_for_bucket.json') as json_file:
    comp1 = json.load(json_file)

In [47]:
class data_comparisons:
    def __init__(self, base, comparison, model_idx):
        self.base_frame_untrimmed = get_rank_frame(base, 0)
        self.comparison_frame_untrimmed  = get_rank_frame(comparison,0)
        self.base_frame, self.comparison_frame = trim_frames(self.base_frame_untrimmed,self.comparison_frame_untrimmed)
        self.base_rank = rank(self.base_frame)
        self.comparison_rank = rank(self.comparison_frame)
        self.metrics = {'top_10_overlap_percent': top_n_overlap_percent(self.base_rank,self.comparison_rank,n=10),
                        'top_20_overlap_percent': top_n_overlap_percent(self.base_rank,self.comparison_rank,n=20),
                        'top_50_overlap_percent': top_n_overlap_percent(self.base_rank,self.comparison_rank,n=50),
                        'ranked_bias_overlap': ranked_bias_overlap(self.base_rank,self.comparison_rank),
                        'kendall_tau': kendall_tau(self.base_rank,self.comparison_rank),
                        'cosine_similarity': cosine_similarity(self.base_frame, self.comparison_frame)
                       }

In [48]:
ob = data_comparisons(base, comp1, 0)

In [49]:
ob.metrics

{'top_10_overlap_percent': 0.7,
 'top_20_overlap_percent': 0.55,
 'top_50_overlap_percent': 0.42,
 'ranked_bias_overlap': 0.540794753836976,
 'kendall_tau': KendalltauResult(correlation=-0.016288900840109522, pvalue=0.40949268826669916),
 'cosine_similarity': 0.217939876375988}

In [25]:
def get_rank_frame(json, model_idx):
    features = json['results'][model_idx]['features']
    coeff = json['results'][model_idx]['coefficients']
    df = pd.DataFrame([features,coeff]).T
    df.columns = ['features','coefficients']
    df['rank'] = abs(df['coefficients'])
    df = df.sort_values(['rank'],ascending = False)
    return df

def trim_frames(df1,df2):
    size_cap = min(len(df1),len(df2))
    df1 = df1.iloc[:size_cap]
    df2 = df2.iloc[:size_cap]
    return df1, df2

def rank(df):
    return list(df['features'])

def ranked_bias_overlap(base_rank, comparator_rank):
    return rbo.RankingSimilarity(base_rank, comparator_rank).rbo()

def top_n_overlap_percent(base_rank, comparator_rank, n=10):
    return len(set(base_rank[0:50]).intersection(set(comparator_rank[0:n]))) / n

from scipy.stats import kendalltau
def kendall_tau(base_rank,comparator_rank):
    return kendalltau(base_rank, comparator_rank)

from scipy.spatial.distance import cosine
def cosine_similarity(base_frame, comparator_frame):
    return 1 - cosine(base_frame['coefficients'].astype(float).to_numpy(),
                  comparator_frame['coefficients'].astype(float).to_numpy())

# def levenstein_distance(base, comparator):
# to be added  